In [29]:
import pygraphviz as pgv
import pandas as pd
import numpy as np

import dowhy
from dowhy import CausalModel
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sys import path
import os
for dirname, _, filenames in os.walk('../resources/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
path.append(os.path.realpath('../'))

from custom import functions

warnings.filterwarnings('ignore')

../resources/allColProSol.xlsx
../resources/dataset_a_2021.csv
../resources/dataset_a_2021c2.csv
../resources/dataset_a_2021v1.xlsx
../resources/dataset_unab_ORIGINAL SIN P1.csv
../resources/dataset_unab_P1.csv
../resources/exitoFallidoEnviosProgramaSol.xlsx
../resources/exitoFallidoProgramaSol.xlsx
../resources/exitosoFallidosEnviosAllColProSol.xlsx
../resources/exitosoFallidosEnviosAllColSol.xlsx
../resources/exitososFallidosEnviosSol.xlsx
../resources/Hito12sinColCeroExitosoFallidosSol.xlsx
../resources/sinColCeroExitosoFallidosEnviosSol.xlsx
../resources/sinColCeroExitosoFallidosSol.xlsx
../resources/v2_hitosExitoFalloColESol1.csv
../resources/v2_hitosExitoFalloColESol1Prograna.csv
../resources/v2_hitosExitoFalloSol1Programa.csv
../resources/causalidad\causalidad.dot
../resources/causalidad\causalidad.png
../resources/causalidad\causalidad2.dot
../resources/causalidad\causalidad_e29.dot
../resources/causalidad\causalidad_exitosos.dot
../resources/causalidad\causalidad_hito1.dot
../

In [30]:
# Read and preview data
df = pd.read_csv("../resources/v2_hitosExitoFalloColESol1.csv", delimiter=";", skipinitialspace=True)

In [31]:
# Convertir la columna "sol1" a números de punto flotante
df['sol1'] = df['sol1'].astype(float)
df['exitosos'] = df['exitosos'].astype(int)
df['fallidos'] = df['fallidos'].astype(int)

In [32]:
#creando columna aprobado y con la funcion set_in_aprobado_nota poblamos la nueva columna.
df['aprobado']=df.apply(lambda x: functions.set_in_aprobado_nota(x['sol1']),axis = 1 )
# df.drop(['hito1', 'hito2', 'exitosos', 'fallidos','sol1','e45', 'e46', 'e47', 'e48', 'e49', 'e50', 'e51', 'e52'], axis=1, inplace=True)
# df.drop(['hito1', 'hito2', 'exitosos', 'fallidos','sol1'], axis=1, inplace=True)

#revisamos la existencia de la nueva columna.
print(df.columns)

Index(['hito1', 'hito2', 'exitosos', 'fallidos', 'e0', 'e1', 'e2', 'e3', 'e4',
       'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12', 'e13', 'e14', 'e15',
       'e16', 'e17', 'e18', 'e19', 'e20', 'e21', 'e22', 'e23', 'e24', 'e25',
       'e26', 'e27', 'e28', 'e29', 'e30', 'e31', 'e32', 'e33', 'e34', 'e35',
       'e36', 'e37', 'e38', 'e39', 'e40', 'e41', 'e42', 'e43', 'e44', 'e45',
       'e46', 'e47', 'e48', 'e49', 'e50', 'e51', 'e52', 'sol1', 'aprobado'],
      dtype='object')


In [33]:
# Crear un objeto del tipo AGraph
graph = pgv.AGraph(directed=True)

In [34]:
# Agregar nodos al gráfico
for column in df.columns:
    graph.add_node(column)

# Agregar aristas al gráfico
for column in df.columns:
    if column != 'hito1' and column != 'aprobado':
        graph.add_edge(column, 'hito1')
        graph.add_edge(column, 'aprobado')

# Guardar el grafo en un archivo DOT en la ruta deseada
graph.write("../resources/causalidad/causalidad_hito1.dot")
# Visualizar el gráfico
graph.layout(prog='dot')
graph.draw('../resources/causalidad/graph_hito1.png')

In [35]:
# Lee el archivo DOT
graph_path = "../resources/causalidad/causalidad_hito1.dot"
model = CausalModel(
    data=df,
    graph=graph_path,
    treatment='hito1',
    outcome='aprobado'
)

In [36]:
# Identificar el estimando causal
identified_estimand = model.identify_effect()

# Estimar el efecto causal utilizando propensity score matching
estimate = model.estimate_effect(
    identified_estimand, method_name="backdoor.propensity_score_matching"
)

# Imprimir el resultado
print(estimate)


propensity_score_matching
*** Causal Estimate ***

## Identified estimand
No directed path from ['hito1'] to ['aprobado'] in the causal graph.
Causal effect is zero.
## Realized estimand
None
## Estimate
Mean value: 0



El grafo causal especifica las siguientes relaciones:

La variable hito1 tiene una flecha dirigida hacia la variable exitosos, lo cual indica que hito1 es una causa directa o un factor que puede influir en exitosos. Esta relación implica que los valores de hito1 pueden afectar los valores de exitosos.

La variable hito1 también tiene una flecha dirigida hacia la variable aprobado, lo cual indica que hito1 es una causa directa o un factor que puede influir en aprobado. Esta relación implica que los valores de hito1 pueden afectar los valores de aprobado.

La variable exitosos también tiene una flecha dirigida hacia la variable aprobado, lo cual indica que exitosos es una causa directa o un factor que puede influir en aprobado. Esta relación implica que los valores de exitosos pueden afectar los valores de aprobado.

In [37]:
#creando graph manual.
causal_graph = """
digraph {
    hito1 -> aprobado;
    hito1 -> e29;
    hito1 -> e28;
    hito1 -> e0;
    e29 -> aprobado;
    e28 -> aprobado;
    e0 -> aprobado;
}
"""

In [38]:
#creando objeto causal
model = CausalModel(data=df, graph=causal_graph, treatment='hito1', outcome='aprobado')


In [39]:
# Identificar el estimando causal utilizando regresión lineal
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
# Estimar el efecto causal utilizando el método de regresión lineal de covariables
estimate = model.estimate_effect(identified_estimand, method_name="backdoor.linear_regression")
# Imprimir el resultado de la estimación
print(estimate)


linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': []}
*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                 
────────(E[aprobado])
d[hito₁]             
Estimand assumption 1, Unconfoundedness: If U→{hito1} and U→aprobado then P(aprobado|hito1,,U) = P(aprobado|hito1,)

## Realized estimand
b: aprobado~hito1
Target units: ate

## Estimate
Mean value: 0.017164481852929236



In [40]:
# Discretizar la variable continua en intervalos
num_bins = 5
df['hito1_bins'] = pd.qcut(df['hito1'], q=num_bins, labels=False, duplicates='drop')

# Crear variables dummy para cada categoría de tratamiento
dummy_variables = pd.get_dummies(df['hito1_bins'], prefix='hito1')

# Combinar las variables dummy con las covariables
covariates = pd.concat([df[['aprobado', 'e29']], dummy_variables], axis=1)

# Ajustar un modelo de regresión múltiple
model = sm.OLS(df['aprobado'], sm.add_constant(covariates))
results = model.fit()

# Obtener los coeficientes de regresión
coefficients = results.params

# Imprimir los coeficientes de regresión
print(coefficients)

const       5.258381e-16
aprobado    1.000000e+00
e29         5.867702e-16
hito1_0    -4.349819e-16
hito1_1    -1.726050e-16
hito1_2    -7.305354e-16
hito1_3    -3.295975e-16
dtype: float64


In [41]:
# Paso 1: Modelar un problema causal
model = CausalModel(
    data=df,
    treatment='e29',  # Variable tratada (exposición)
    outcome='aprobado',  # Variable de resultado
    common_causes=['e0', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8', 'e9', 'e10',
       'e11', 'e12', 'e13', 'e14', 'e15', 'e16', 'e17', 'e18', 'e19', 'e20',
       'e21', 'e22', 'e23', 'e24', 'e25', 'e26', 'e27', 'e28', 'e30',
       'e31', 'e32', 'e33', 'e34', 'e35', 'e36', 'e37', 'e38', 'e39', 'e40',
       'e41', 'e42', 'e43', 'e44', 'e45', 'e46', 'e47', 'e48', 'e49', 'e50',
       'e51', 'e52']  # Variables de causa común
)

In [42]:
# Paso 2: Identificar el estimando objetivo bajo el modelo
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

In [43]:
# Paso 3: Estimar el efecto causal basado en el estimando identificado
estimate = model.estimate_effect(identified_estimand,
                                 method_name='backdoor.propensity_score_matching')
print(estimate)

propensity_score_matching
*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                                                                           
──────(E[aprobado|e50,e37,e11,e9,e12,e18,e2,e17,e24,e49,e47,e22,e51,e25,e30,e4
d[e₂₉]                                                                        

                                                                              
2,e31,e3,e48,e36,e15,e5,e43,e44,e35,e27,e32,e7,e1,e13,e23,e38,e0,e52,e16,e39,e
                                                                              

                                                             
41,e21,e14,e6,e28,e10,e4,e20,e34,e8,e19,e26,e46,e33,e40,e45])
                                                             
Estimand assumption 1, Unconfoundedness: If U→{e29} and U→aprobado then P(aprobado|e29,e50,e37,e11,e9,e12,e18,e2,e17,e24,e49,e47,e22,e51,e25,e30,e42,e31,e3,e48,e36,e15,

In [44]:
# Paso 4: Refutar el estimado obtenido
refute_results = model.refute_estimate(identified_estimand, estimate,
                                       method_name='random_common_cause')

In [45]:
# Resultados
print(estimate)
print(refute_results)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                                                                           
──────(E[aprobado|e50,e37,e11,e9,e12,e18,e2,e17,e24,e49,e47,e22,e51,e25,e30,e4
d[e₂₉]                                                                        

                                                                              
2,e31,e3,e48,e36,e15,e5,e43,e44,e35,e27,e32,e7,e1,e13,e23,e38,e0,e52,e16,e39,e
                                                                              

                                                             
41,e21,e14,e6,e28,e10,e4,e20,e34,e8,e19,e26,e46,e33,e40,e45])
                                                             
Estimand assumption 1, Unconfoundedness: If U→{e29} and U→aprobado then P(aprobado|e29,e50,e37,e11,e9,e12,e18,e2,e17,e24,e49,e47,e22,e51,e25,e30,e42,e31,e3,e48,e36,e15,e5,e43,e44,e35,e27,e32,e7,